In [ ]:
import requests
import msal
import csv
import pandas as pd
from pyspark.sql import SparkSession

In [ ]:
# ----------- CONFIGURATION ----------- #
client_id = '<YOUR_CLIENT_ID>'
client_secret = '<YOUR_CLIENT_SECRET>'
tenant_id = '<YOUR_TENANT_ID>'
workspace_id = '<YOUR_WORKSPACE_ID>'

authority = f"https://login.microsoftonline.com/{tenant_id}"
scope = ["https://analysis.windows.net/powerbi/api/.default"]
api_base = "https://api.powerbi.com/v1.0/myorg"
api_base1 = "https://api.fabric.microsoft.com/v1/workspaces"
#"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks"

# ----------- AUTHENTICATION ----------- #
app = msal.ConfidentialClientApplication(
    client_id, authority=authority, client_credential=client_secret
)
token_result = app.acquire_token_for_client(scopes=scope)
access_token = token_result['access_token']
headers = {"Authorization": f"Bearer {access_token}"}
fabric_artifacts = {"notebooks", "dataPipelines"}

# ----------- HELPER FUNCTION ----------- #
def get_artifacts(endpoint, artifact_type):
    if artifact_type in fabric_artifacts:
        response = requests.get(f"{api_base1}/{workspace_id}/{endpoint}", headers=headers)
    else:
        response = requests.get(f"{api_base}/groups/{workspace_id}/{endpoint}", headers=headers)

    response.raise_for_status()
    data = response.json().get('value', [])

    if artifact_type in fabric_artifacts:
        return [
            {
                "artifact_type": artifact_type,
                "name": item.get("displayName"),
                "id": item.get("id"),
                "webUrl": item.get("webUrl", ""),
                "createdBy": item.get("createdBy", {}).get("email", "")
            } 
            for item in data
            ]
    else:    
        return [
            {
                "artifact_type": artifact_type,
                "name": item.get("name"),
                "id": item.get("id"),
                "webUrl": item.get("webUrl", ""),
                "createdBy": item.get("createdBy", {}).get("email", "")
            }
            for item in data
        ]

# ----------- FETCH ARTIFACTS ----------- #
all_artifacts = []
all_artifacts += get_artifacts("datasets", "Dataset")
all_artifacts += get_artifacts("reports", "Report")
all_artifacts += get_artifacts("dashboards", "Dashboard")
all_artifacts += get_artifacts("dataflows", "Dataflow")
all_artifacts += get_artifacts("notebooks", "notebooks")
all_artifacts += get_artifacts("dataPipelines", "dataPipelines")
# all_artifacts += get_artifacts("scorecards", "Scorecard")  # optional

# ----------- CREATE DATAFRAME ----------- #
df_artifacts = pd.DataFrame(all_artifacts)

# ----------- DISPLAY DATA ----------- #
print(df_artifacts)
spark_df = spark.createDataFrame(df_artifacts)
spark_df = spark.createDataFrame(df_artifacts.astype(str))
spark_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("power_bi_artifacts")
